In [1]:
!pip install opendatasets --quiet
import opendatasets as od
od.download('https://www.kaggle.com/datasets/emmarex/plantdisease')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


 GeorgeGiosmas
Your Kaggle Key: ········
Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease


100%|████████████████████████████████████████████████████████████████████████████████| 658M/658M [00:01<00:00, 551MB/s]


In [2]:
import torch
from torch import nn
from torch.optim import Adam
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import os

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [9]:
root_path = 'plantdisease/PlantVillage/'
img_path = []
labels_path = []

for label in os.listdir(root_path):
    for item in os.listdir(f'{root_path}/{label}'):
        img_path.append(f'{root_path}/{label}/{item}')
        labels_path.append(label)
        
print(f'Number of Images: {len(img_path)}')

Number of Images: 20639


In [15]:
# Creation of our Dataframe
data_df = pd.DataFrame(zip(img_path, labels_path), columns = ['image_path', 'label'])

# Print the distribution of data among classes and the format of our DataFrame.
print(data_df['label'].value_counts())
data_df.head()

label
Tomato__Tomato_YellowLeaf__Curl_Virus          3209
Tomato_Bacterial_spot                          2127
Tomato_Late_blight                             1909
Tomato_Septoria_leaf_spot                      1771
Tomato_Spider_mites_Two_spotted_spider_mite    1676
Tomato_healthy                                 1591
Pepper__bell___healthy                         1478
Tomato__Target_Spot                            1404
Potato___Early_blight                          1000
Potato___Late_blight                           1000
Tomato_Early_blight                            1000
Pepper__bell___Bacterial_spot                   997
Tomato_Leaf_Mold                                952
Tomato__Tomato_mosaic_virus                     373
Potato___healthy                                152
Name: count, dtype: int64


,image_path,label
0,plantdisease/PlantVillage//Pepper__bell___Bact...,Pepper__bell___Bacterial_spot
1,plantdisease/PlantVillage//Pepper__bell___Bact...,Pepper__bell___Bacterial_spot
2,plantdisease/PlantVillage//Pepper__bell___Bact...,Pepper__bell___Bacterial_spot
3,plantdisease/PlantVillage//Pepper__bell___Bact...,Pepper__bell___Bacterial_spot
4,plantdisease/PlantVillage//Pepper__bell___Bact...,Pepper__bell___Bacterial_spot


In [20]:
train = data_df.sample(frac=0.8)
val = data_df.drop(train.index)
test = val.sample(frac=0.5)
val = val.drop(test.index)

print(f'Train size: {len(train)}, Validation size: {len(val)}, Test size: {len(test)}')

Train size: 16511, Validation size: 2064, Test size: 2064


In [26]:
# Create a LabelEncoder for the Labels
label_encoder = LabelEncoder()
label_encoder.fit(data_df['label'])

# Create a transform for transforming the images in the same - appropriate form
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.ConvertImageDtype(dtype=float)
])

In [ ]:
class PlantsDataset(dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.labels = torch.tesnor(label_encoder.transform(dataframe['label'])).to(device)
        
    def __len__(self):
        return self.dataframe.shape[0]
    
    def __getitem__(self, indx):
        
        
        return image, label